In [108]:
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
from bs4 import BeautifulSoup
import urllib.request
import re
import requests
import pandas as pd
import datetime
import os
import glob


In [109]:

df = pd.DataFrame(columns=['ilosc_przejazdow', 'data_przejazdu'])

html_page = requests.get("https://www.wroclaw.pl/open-data/dataset/wrmprzejazdy_data/resource_history/c737af89-bcf7-4f7d-8bbc-4a0946d7006e").text
soup = BeautifulSoup(html_page, 'html.parser')
list = soup.find_all(class_='heading')
list = list[1:]
list.reverse()

In [110]:
date2022 = datetime.datetime(2022, 1, 1)

In [122]:

# list = list[:5] #test na malej ilosc elem
# quantity = []
for elem in list:

    name = elem.contents
    elem = str(elem)

    name = str(name)
    name = name.split("_")[2]

    date = datetime.datetime(int(name.split("-")[0]), int(name.split("-")[1]), int(name.split("-")[2]))
    if date>=date2022:
        link = elem.split(" ")
        link = link[2]
        link = link[5:]
        link = link[1:]
        link = link[:-1]
        # link = link.replace("\""," ")

        if os.path.exists(os.path.join("daily_csv", f"{name}.csv")):
            dailyDate = pd.read_csv(os.path.join("daily_csv", f"{name}.csv"))
            quantity = len(dailyDate)
            df = df.append({'ilosc_przejazdow':quantity,'data_przejazdu':name},ignore_index=True)
            continue

        dailyDate = pd.read_csv(link)
        dailyDate.to_csv(os.path.join("daily_csv", f"{name}.csv"))
        quantity = len(dailyDate)
        df = df.append({'ilosc_przejazdow':quantity,'data_przejazdu':name},ignore_index=True)



        # df["ilosc_przejazdow"]=c.__len__()
        # print(str(date) + " greater than "+ str(date2022))
    # else : print(str(date) + " less than "+ str(date2022))


# df["ilosc_przejazdow"]=quantity
df
# items = list.find_all('a')
# print(soup.prettify())
# for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
#     print(link.get('href'))

C:\Users\march\AppData\Local\Temp\ipykernel_10208\2380825751.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\march\AppData\Local\Temp\ipykernel_10208\2380825751.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\march\AppData\Local\Temp\ipykernel_10208\2380825751.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\march\AppData\Local\Temp\ipykernel_10208\2380825751.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\march\AppData\Local\Temp\ipykernel_10208\2380825751.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

,ilosc_przejazdow,data_przejazdu
0,1259,2022-1-1
1,1259,2022-1-2
2,1172,2022-1-3
3,2083,2022-1-4
4,2083,2022-1-4
...,...,...
475,11328,2022-6-2
476,11309,2022-6-3
477,11299,2022-6-4
478,11777,2022-6-5


In [112]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["data_przejazdu"],
                         y=df["ilosc_przejazdow"],
                         mode='markers',
                         marker=dict(size=3, color="blue")))
fig.show()

In [113]:

all_files = glob.glob("root_csv"+"/*.csv")
archieved = pd.concat(map(pd.read_csv, all_files), ignore_index=True)
archieved = pd.DataFrame(archieved, columns=["UID wynajmu","Data zwrotu"])
archieved["Data zwrotu"] = pd.to_datetime(archieved["Data zwrotu"]).dt.date
archieved= archieved.groupby(archieved["Data zwrotu"]).count()
archieved = archieved.reset_index()

In [114]:
archieved["ilosc_przejazdow"] = archieved["UID wynajmu"]
archieved["data_przejazdu"] = archieved["Data zwrotu"]
# archieved.rename(columns={'UID wynajmu': 'przejazdy'})
archieved


,Data zwrotu,UID wynajmu,ilosc_przejazdow,data_przejazdu
0,2020-01-01,634,634,2020-01-01
1,2020-01-02,971,971,2020-01-02
2,2020-01-03,1017,1017,2020-01-03
3,2020-01-04,492,492,2020-01-04
4,2020-01-05,629,629,2020-01-05
...,...,...,...,...
665,2021-11-26,3663,3663,2021-11-26
666,2021-11-27,3158,3158,2021-11-27
667,2021-11-28,1191,1191,2021-11-28
668,2021-11-29,2707,2707,2021-11-29


In [115]:
all_time = pd.concat([archieved, df], ignore_index=True).drop(["Data zwrotu", "UID wynajmu"], axis=1)
all_time["data_przejazdu"] = pd.to_datetime(all_time["data_przejazdu"], format='%Y-%m-%d').dt.date
all_time.set_index("data_przejazdu", inplace=True)
all_time

,ilosc_przejazdow
data_przejazdu,
2020-01-01,634
2020-01-02,971
2020-01-03,1017
2020-01-04,492
2020-01-05,629
...,...
2022-06-02,11328
2022-06-03,11309
2022-06-04,11299


In [116]:
import meteostat
# from datetime import datetime, timedelta
# Przygotowanie obiektów dat
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime.now()

# Pobieranie danych nt. stacji
stations = meteostat.Stations()
wroclaw_req = stations.nearby(51, 17)
wroclaw = wroclaw_req.fetch(1)

# Pobieranie danych ze stacji day by day
weather_data_req = meteostat.Daily(wroclaw, start, end)
weather_data = weather_data_req.fetch()
weather_data = weather_data.reset_index()
weather_data = weather_data[["time", "tavg", "prcp"]]
weather_data["time"] = pd.to_datetime(weather_data["time"]).dt.date
weather_data.set_index("time", inplace=True)
weather_data


,tavg,prcp
time,,
2020-01-01,1.3,0.0
2020-01-02,-2.2,0.0
2020-01-03,1.1,0.0
2020-01-04,4.3,0.5
2020-01-05,0.3,1.0
...,...,...
2022-06-01,16.5,0.0
2022-06-02,15.2,0.3
2022-06-03,17.3,0.0


In [124]:
# merged = pd.concat([all_time, weather_data],  join="inner", axis=1)
merged = all_time.join(weather_data)
merged = merged.fillna(0)
merged


,ilosc_przejazdow,tavg,prcp
2020-01-01,634,1.3,0.0
2020-01-02,971,-2.2,0.0
2020-01-03,1017,1.1,0.0
2020-01-04,492,4.3,0.5
2020-01-05,629,0.3,1.0
...,...,...,...
2022-06-02,11328,15.2,0.3
2022-06-03,11309,17.3,0.0
2022-06-04,11299,19.7,3.5
2022-06-05,11777,19.1,0.0


In [123]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=merged.index,
                         y=merged["ilosc_przejazdow"],
                         mode='markers',
                         marker=dict(size=3, color="blue")
                         ))
fig.show()